In [ ]:
# necessary import functions
import time
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
import base64
from IPython.display import HTML

from pyspark.sql import functions as F
from sedona.register import SedonaRegistrator
SedonaRegistrator.registerAll(spark)

In [ ]:
# Port of Djibouti, Djibouti	11.5890 N	43.1457 E
# Port Said, Egypt	31.2565 N	32.2849 E
# Port of Alexandria, Egypt	31.2001 N	29.9187 E
# Entry Point of Suez Canal, Egypt	30.5852 N	32.2650 E
# Exit Point of Suez Canal, Egypt	29.9636 N	32.5618 E
# Port of Mombasa, Kenya	4.0435 S	39.6668 E
# Ports of Madagascar (Toamasina), Madagascar	18.1429 S	49.4080 E
# Ports of Beira, Mozambique	19.8286 S	34.8385 E
# Ports of Nacala, Mozambique	14.5519 S	40.6725 E
# Ports of Maputo, Mozambique	25.9655 S	32.5892 E
# Port Sudan, Sudan	19.5903 N	37.2080 E
# Port of Dar es-Salaam, Tanzania	6.7924 S	39.2083 E
# Chornomorsk,port,UKR,46.3429225,30.6452624
# Odesa,port,UKR,46.489814,30.7579897
# Yuzhny/Pivdenny,port,UKR,46.6189212,31.0267066
# Bosphorus,strait,TUR,41.11833286,29.07183305
# Novorossiysk,port,RUS,44.7252759,37.7760587
# Sevastopol,port,RUS,44.6216741,33.52471752
# Ust-Luga,port,RUS,59.67942354,28.40689607
# Vostochny,port,RUS,42.75240645,133.0568527
# Primorsk,port,RUS,60.33400206,28.71043499
# Murmansk,port,RUS,68.97518069,33.05367146
# Taman,port,RUS,45.12601929,36.68152034
# St. Petersburg,port,RUS,59.87974862,30.19520534
# Vanino,port,RUS,49.08644498,140.2727319
# Vladivostok,port,RUS,43.0896029,131.8748952
# Sabetta,port,RUS,71.27891484,72.09328074

In [ ]:
# big bbox
# latitude min -30 max 32
# longitude min 20 max 60

# small bbox
# latitude min 39 max 50
# longitude min 25 max 45

In [ ]:
# helper functions
# save a local CSV from the notebook
def create_download_link(query, title="Download CSV file", filename="data.csv"):
    start_time = time.monotonic()
    df = query.toPandas()
    csv = df.to_csv()
    # with open(filename, "w", encoding="utf-8") as fout:
    #     print(csv, file=fout)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload, title=title, filename=filename)
    display(f"{time.monotonic() - start_time}s")
    return HTML(html)

# read in UNGP S3 data from a range of dates
def get_date_list(basepath, start_date, end_date):
    start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d").date()
    end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d").date()
    delta = end_date - start_date
    days = []
    for i in range(delta.days + 1):
        day = start_date + datetime.timedelta(days=i)
        days.append(datetime.datetime.strftime(day, "%Y-%m-%d"))
    
    paths = [basepath + f"year={day[:4]}/month={day[5:7]}/day={day[8:10]}" for day in days]
    return (paths)

In [ ]:
# geographic locations
# locations = pd.read_csv("https://github.com/dhopp1-UNCTAD/ais_helper_files/raw/main/geographic_locations.csv")
locations = pd.read_csv("https://raw.githubusercontent.com/UNECE/AIS/master/wpi_12nm_bounding_box_port.csv")

In [ ]:
# countries = ["UA", "DJ", "EG", "KE", "MZ", "SD", "TZ", "RU"]
# countries = ["UA", "RU"]
countries = ["DJ", "EG", "KE", "MZ", "SD", "TZ"]

In [ ]:
locations.columns

In [ ]:
" ".join(sorted([f"{cnt}" for cnt in locations["COUNTRY"].unique()]))

In [ ]:
locations.loc[locations["COUNTRY"].isna()]  # both are in Namibia I think pandas converted NA into NaN even though it is a str

In [ ]:
locs = locations.loc[locations["COUNTRY"].isin(countries)][["PORT_NAME", "COUNTRY", "LATITUDE", "LONGITUDE"]].reset_index(drop=True)
locs

In [ ]:
set(locs["COUNTRY"].unique()) == set(countries)

In [ ]:
locs["name"] = locs["PORT_NAME"] + " (" + locs["COUNTRY"] + ")"

In [ ]:
locs["longitude"] = locs["LONGITUDE"]
locs["latitude"] = locs["LATITUDE"]

In [ ]:
locs

In [ ]:
def get_data(start_date, end_date, locations, distance_parameter = "0.3"):
    # distance parameter = 0.01 = 1 kilometer radius
    # distance parameter = 0.3 = 30 kilometer radius?
    
    # big bbox (africa)
    # latitude min -30 max 32
    # longitude min 20 max 60
    bbox_lat_min = -30
    bbox_lat_max = 32
    bbox_lon_min = 20
    bbox_lon_max = 60
    
    # small bbox (black sea)
    # latitude min 39 max 50
    # longitude min 25 max 45
    # bbox_lat_min = 39
    # bbox_lat_max = 50
    # bbox_lon_min = 25
    # bbox_lon_max = 45

    # all geographies in one query
    condition_string = ""
    select_string = ""
    pos = "pos"
    for name_i in locations.name:
        name_s = name_i.replace('\'', '')
        condition_string += f"""ST_Contains(ST_Buffer(ST_Point({locations.loc[locations.name == name_i, 'longitude'].values[0]}, {locations.loc[locations.name == name_i, 'latitude'].values[0]}), {distance_parameter}), {pos})"""
        if name_i != locations.name.values[-1]:
            condition_string += " OR "
        if name_i == locations.name.values[0]:
            select_string += f"""CASE WHEN ST_Contains(ST_Buffer(ST_Point({locations.loc[locations.name == name_i, 'longitude'].values[0]}, {locations.loc[locations.name == name_i, 'latitude'].values[0]}), {distance_parameter}), {pos}) THEN '{name_s}' """
        elif name_i != locations.name.values[-1]:
            select_string += f"""WHEN ST_Contains(ST_Buffer(ST_Point({locations.loc[locations.name == name_i, 'longitude'].values[0]}, {locations.loc[locations.name == name_i, 'latitude'].values[0]}), {distance_parameter}), {pos}) THEN '{name_s}' """
        else:
            select_string += f"""WHEN ST_Contains(ST_Buffer(ST_Point({locations.loc[locations.name == name_i, 'longitude'].values[0]}, {locations.loc[locations.name == name_i, 'latitude'].values[0]}), {distance_parameter}), {pos}) THEN '{name_s}' """
            select_string += "END AS geo_name"
    
    # step 1
    # read data
    basepath = "s3a://ungp-ais-data-historical-backup/exact-earth-data/transformed/prod/"
    dates = get_date_list(basepath, start_date, end_date)
    df = spark.read.parquet(*dates)

    # create temp view to be able to use spark SQL
    df.createOrReplaceTempView("df")
    
    # print(spark.sql("SELECT * FROM df LIMIT 1").toPandas().values.tolist())
    # print(spark.sql("SELECT * FROM df LIMIT 1").toPandas().columns.tolist())

    # adding points and filtering for cargo and tankers
    step_01 = spark.sql(f"""
                    SELECT DISTINCT vessel_type, mmsi, date_year, date_month, {select_string} FROM
                    (
                    SELECT vessel_type, mmsi, date_year, date_month, ST_Point(lon, lat) as pos FROM
                    (
                        SELECT DISTINCT
                            YEAR(dt_pos_utc) as date_year, 
                            MONTH(dt_pos_utc) AS date_month,
                            mmsi, 
                            vessel_type,
                            cast(longitude as Decimal(6,3)) as lon,
                            cast(latitude as Decimal(6,3)) as lat
                        FROM df
                        WHERE vessel_type IN ('Cargo','Tanker') AND (nav_status_code = 1 or nav_status_code = 5)
                    ) AS subquery
                    WHERE lon >= {bbox_lon_min} AND lon <= {bbox_lon_max} AND lat >= {bbox_lat_min} AND lat <= {bbox_lat_max}
                    ) AS subsubquery
                    WHERE {condition_string}
                    """)
    return (step_01)

In [ ]:
# queries for months
# start_month = datetime.datetime.strptime("2018-12-01", "%Y-%m-%d")
# end_month = datetime.datetime.strptime("2020-02-01", "%Y-%m-%d")
# start_month = datetime.datetime.strptime("2020-03-01", "%Y-%m-%d")
# end_month = datetime.datetime.strptime("2021-02-01", "%Y-%m-%d")
# start_month = datetime.datetime.strptime("2021-03-01", "%Y-%m-%d")
# end_month = datetime.datetime.strptime("2021-07-01", "%Y-%m-%d")
# start_month = datetime.datetime.strptime("2021-08-01", "%Y-%m-%d")
# end_month = datetime.datetime.strptime("2022-10-01", "%Y-%m-%d")
start_month = datetime.datetime.strptime("2022-11-01", "%Y-%m-%d")
end_month = datetime.datetime.strptime("2023-10-01", "%Y-%m-%d")

start_dates = []
end_dates = []

while start_month <= end_month:
    start_dates.append(datetime.datetime.strftime(start_month, "%Y-%m-%d"))
    end_date = min(start_month + relativedelta(months=1) - relativedelta(days = 1), datetime.datetime.today() - relativedelta(days=2)) # minimum between 2 days ago so don't go ahead of where there are actually files
    end_dates.append(datetime.datetime.strftime(end_date, "%Y-%m-%d"))
    start_month = start_month + relativedelta(months=1)

date_dict = {f"{x}": None for x in start_dates}

for i in range(len(start_dates)):
    date_dict[f"{start_dates[i]}"] = get_data(start_dates[i], end_dates[i], locs)

In [ ]:
# prefix = "rus_ukr_ships"
prefix = "black_sea_horn_africa_ships"

In [ ]:
quick = ["2022-01-01", "2022-02-01", "2022-03-01"]

In [ ]:
# for date_str in quick:
#     display((prefix, date_str))
#     display(create_download_link(date_dict[date_str], filename=f"{prefix}_{date_str}.csv"))

In [ ]:
for date_str, query in date_dict.items():
    if date_str in quick:
        continue
    display((prefix, date_str))
    display(create_download_link(query, filename=f"{prefix}_{date_str}.csv"))

In [ ]:
countries = ["UA", "DJ", "EG", "KE", "MZ", "SD", "TZ", "RU", "ER", "SO", "YE"]
locations = pd.read_csv("https://raw.githubusercontent.com/UNECE/AIS/master/wpi_12nm_bounding_box_port.csv")
locs = locations.loc[locations["COUNTRY"].isin(countries)][["PORT_NAME", "COUNTRY", "LATITUDE", "LONGITUDE"]].reset_index(drop=True)
locs["name"] = locs["PORT_NAME"] + " (" + locs["COUNTRY"] + ")"
locs["longitude"] = locs["LONGITUDE"]
locs["latitude"] = locs["LATITUDE"]
locs

In [ ]:
s_locations = pd.DataFrame({
    "name": [
        "Suez (EG)",
        "Bosphorus (TR)",
    ],
    "longitude": [
        32.355877,
        29.07183305,
    ],
    "latitude": [
        30.443370,
        41.11833286,
    ],
})

In [ ]:
llocs = pd.concat([locs, s_locations]).reset_index(drop=True)

In [ ]:
lcsv = llocs[["name", "longitude", "latitude"]].to_csv(index=False)
lb64 = base64.b64encode(lcsv.encode())
lpayload = lb64.decode()
lhtml = f'<a download="locs.csv" href="data:text/csv;base64,{lpayload}" target="_blank">locs.csv</a>'
display(HTML(lhtml))